In [73]:
!pip install -U sentence-transformers
!pip install chromadb
!pip install openai


^C


In [11]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [14]:
df = pd.read_csv('/content/imdb_top_1000.csv')

In [15]:
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [16]:
df['text'] = df.apply(lambda x : x['Overview'] + ' ' + x['Director'] + ' ' + x['Star1'] + ' ' + x['Star2'] + ' ' + x['Star3'] + ' ' + x['Star4'], axis=1)

In [17]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [18]:
embeddings = model.encode(df['text'], batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [19]:
df['embeddings'] = embeddings.tolist()

In [20]:
df['ids'] = df.index
df['ids'] = df['ids'].astype(str)

In [21]:
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text,embeddings,ids
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...,"[-0.11102541536092758, -0.00945297721773386, -...",0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...,"[-0.10081297904253006, -0.021370679140090942, ...",1
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...,"[0.005191366653889418, 0.042379382997751236, -...",2
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...,"[-0.0728299617767334, 0.007155474741011858, -0...",3
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...,"[-0.08513770997524261, 0.057809703052043915, -...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,A young New York socialite becomes interested ...,"[0.028408022597432137, -0.029192084446549416, ...",995
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,Sprawling epic covering the life of a Texas ca...,"[0.02407665364444256, -0.03670534864068031, 0....",996
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","In Hawaii in 1941, a private is cruelly punish...","[-0.08005382120609283, 0.05756460130214691, -0...",997
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,Several survivors of a torpedoed merchant ship...,"[-0.020807666704058647, 0.05352263152599335, -...",998


#### Chroma

In [22]:
import chromadb
from chromadb.utils import embedding_functions

In [23]:
# for OpenAI embeddings
# openai_ef = embedding_functions.OpenAIEmbeddingFunction(
#     api_key='xxx',
#     model_name = 'text-embedding-ada-002'
# )

# No es necesario porque chroma utiliza por sentece transformers como default
sentence_transformers_ef = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name = 'sentence-transformers/all-MiniLM-L6-v2'
)



In [26]:
chroma_client = chromadb.Client()
client_persistent = chromadb.PersistentClient(path='/content/data_embeddings')

In [27]:
db = client_persistent.create_collection(name='movies_db', embedding_function=sentence_transformers_ef)

In [28]:
db.add(
    ids = df['ids'].tolist(),
    embeddings = df['embeddings'].tolist(),
    metadatas=df.drop(['ids', 'embeddings', 'text'], axis=1).to_dict('records')
)

In [74]:
db.peek(1)

{'ids': ['0'],
 'embeddings': [[-0.11102541536092758,
   -0.00945297721773386,
   -0.061688411980867386,
   -0.0182977132499218,
   -0.009833157993853092,
   0.06417812407016754,
   0.08203236758708954,
   -0.04906788468360901,
   0.06276262551546097,
   -0.03238978981971741,
   0.012614275328814983,
   0.03933822736144066,
   -0.025924425572156906,
   0.022846553474664688,
   -0.025803297758102417,
   -0.03067529946565628,
   -0.0682935118675232,
   0.03191167116165161,
   0.025044355541467667,
   0.07082391530275345,
   0.0062096272595226765,
   -0.030562840402126312,
   -0.016868935897946358,
   -0.037087272852659225,
   0.004426752217113972,
   0.032468803226947784,
   -0.010899782180786133,
   -0.05131009221076965,
   -0.0036282476503401995,
   0.0382995642721653,
   0.028951948508620262,
   0.014759032987058163,
   0.10134254395961761,
   0.04917678236961365,
   0.06850536912679672,
   0.021276874467730522,
   0.007060567382723093,
   0.02818995527923107,
   0.05037762597203255,


#### Chroma Embeddings

In [34]:
db_no_embeddings = client_persistent.create_collection(name='movies_db_noo_embeddings', embedding_function=sentence_transformers_ef)


In [36]:
db_no_embeddings.add(
    ids=df['ids'].tolist(),
    documents=df['text'].tolist(),
    metadatas=df.drop(['ids', 'embeddings', 'text'], axis=1).to_dict('records')
)

In [37]:
db_no_embeddings.peek(1)


{'ids': ['0'],
 'embeddings': [[-0.11102540791034698,
   -0.009452952072024345,
   -0.06168840453028679,
   -0.018297765403985977,
   -0.009833198972046375,
   0.06417813897132874,
   0.08203230053186417,
   -0.049067895859479904,
   0.06276260316371918,
   -0.032389793545007706,
   0.01261431910097599,
   0.03933827206492424,
   -0.025924403220415115,
   0.02284664288163185,
   -0.02580324374139309,
   -0.03067522495985031,
   -0.0682935118675232,
   0.03191167488694191,
   0.02504432387650013,
   0.07082392275333405,
   0.006209616083651781,
   -0.030562713742256165,
   -0.01686890423297882,
   -0.037087325006723404,
   0.004426705185323954,
   0.032468777149915695,
   -0.010899816639721394,
   -0.051310181617736816,
   -0.0036282341461628675,
   0.038299497216939926,
   0.02895197831094265,
   0.014759044162929058,
   0.1013425961136818,
   0.04917675256729126,
   0.06850545853376389,
   0.02127692848443985,
   0.0070604667998850346,
   0.02818991243839264,
   0.050377678126096725,


#### Chroma Query

In [39]:
results = db_no_embeddings.query(
    query_texts = ['a history with elves and a ring'],
    n_results = 2
)

In [42]:
results['metadatas'][0][1]

{'Certificate': 'U',
 'Director': 'Peter Jackson',
 'Genre': 'Action, Adventure, Drama',
 'Gross': '315,544,750',
 'IMDB_Rating': 8.8,
 'Meta_score': 92.0,
 'No_of_Votes': 1661481,
 'Overview': 'A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.',
 'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BN2EyZjM3NzUtNWUzMi00MTgxLWI0NTctMzY4M2VlOTdjZWRiXkEyXkFqcGdeQXVyNDUzOTQ5MjY@._V1_UX67_CR0,0,67,98_AL_.jpg',
 'Released_Year': '2001',
 'Runtime': '178 min',
 'Series_Title': 'The Lord of the Rings: The Fellowship of the Ring',
 'Star1': 'Elijah Wood',
 'Star2': 'Ian McKellen',
 'Star3': 'Orlando Bloom',
 'Star4': 'Sean Bean'}

#### Where

1. Estructura

```
{ "metadata_field" : { <Operator>: <Value> } }
```

2. Operadores
```
$eq - equal to (string, int, float)
$ne - not equal to (string, float)
$gt - greater than (int, float)
$gte - greater than or equal to (int, float)
$lt - less than(int, float)
$lte - less than or equal to (int, float)
```

In [63]:
results_where = db_no_embeddings.query(
    query_texts = ['a history with elves and a ring'],
    n_results = 2,
    where = {
        'IMDB_Rating': {'$gte': 8}
    }
)

In [64]:
results_where

{'ids': [['10', '5']],
 'distances': [[1.0577359199523926, 1.1908358335494995]],
 'metadatas': [[{'Certificate': 'U',
    'Director': 'Peter Jackson',
    'Genre': 'Action, Adventure, Drama',
    'Gross': '315,544,750',
    'IMDB_Rating': 8.8,
    'Meta_score': 92.0,
    'No_of_Votes': 1661481,
    'Overview': 'A meek Hobbit from the Shire and eight companions set out on a journey to destroy the powerful One Ring and save Middle-earth from the Dark Lord Sauron.',
    'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BN2EyZjM3NzUtNWUzMi00MTgxLWI0NTctMzY4M2VlOTdjZWRiXkEyXkFqcGdeQXVyNDUzOTQ5MjY@._V1_UX67_CR0,0,67,98_AL_.jpg',
    'Released_Year': '2001',
    'Runtime': '178 min',
    'Series_Title': 'The Lord of the Rings: The Fellowship of the Ring',
    'Star1': 'Elijah Wood',
    'Star2': 'Ian McKellen',
    'Star3': 'Orlando Bloom',
    'Star4': 'Sean Bean'},
   {'Certificate': 'U',
    'Director': 'Peter Jackson',
    'Genre': 'Action, Adventure, Drama',
    'Gross': '377,845,90

In [75]:
results_where = db_no_embeddings.query(
    query_texts = ['a history with elves and a ring'],
    n_results = 2,
    where = { "$and":[ {
              'IMDB_Rating' : {
                  "$gte": 8
              }
    },
              {
              'Released_Year': {
                  "$ne":'2001'
              }
    }

                    ]}
)

In [76]:
results_where

{'ids': [['5', '13']],
 'distances': [[1.1908358335494995, 1.2519809007644653]],
 'metadatas': [[{'Certificate': 'U',
    'Director': 'Peter Jackson',
    'Genre': 'Action, Adventure, Drama',
    'Gross': '377,845,905',
    'IMDB_Rating': 8.9,
    'Meta_score': 94.0,
    'No_of_Votes': 1642758,
    'Overview': "Gandalf and Aragorn lead the World of Men against Sauron's army to draw his gaze from Frodo and Sam as they approach Mount Doom with the One Ring.",
    'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BNzA5ZDNlZWMtM2NhNS00NDJjLTk4NDItYTRmY2EwMWZlMTY3XkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UX67_CR0,0,67,98_AL_.jpg',
    'Released_Year': '2003',
    'Runtime': '201 min',
    'Series_Title': 'The Lord of the Rings: The Return of the King',
    'Star1': 'Elijah Wood',
    'Star2': 'Viggo Mortensen',
    'Star3': 'Ian McKellen',
    'Star4': 'Orlando Bloom'},
   {'Certificate': 'UA',
    'Director': 'Peter Jackson',
    'Genre': 'Action, Adventure, Drama',
    'Gross': '342,551,365',

#### Pinecone

In [77]:
!pip install pinecone-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.4/179.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 18.4 MB/s eta 0:00:00


In [78]:
import pinecone
from getpass import getpass

In [79]:
pinecone_api = getpass('Enter th esecret value: ')

Enter th esecret value: ··········


In [80]:
pinecone.init(api_key=pinecone_api, environment="us-west1-gcp")

In [82]:
dimension_embeddings = len(df['embeddings'][0])
dimension_embeddings

384

#### Create Index

In [83]:
pinecone.create_index('movies-embeddings', dimension=dimension_embeddings, metric='cosine')

In [84]:
index = pinecone.Index('movies-embeddings')

#### Ingest data

In [86]:
df = df.fillna(' ')
df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,text,embeddings,ids
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Two imprisoned men bond over a number of years...,"[-0.11102541536092758, -0.00945297721773386, -...",0
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",An organized crime dynasty's aging patriarch t...,"[-0.10081297904253006, -0.021370679140090942, ...",1
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",When the menace known as the Joker wreaks havo...,"[0.005191366653889418, 0.042379382997751236, -...",2
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",The early life and career of Vito Corleone in ...,"[-0.0728299617767334, 0.007155474741011858, -0...",3
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",A jury holdout attempts to prevent a miscarria...,"[-0.08513770997524261, 0.057809703052043915, -...",4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,,A young New York socialite becomes interested ...,"[0.028408022597432137, -0.029192084446549416, ...",995
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,,Sprawling epic covering the life of a Texas ca...,"[0.02407665364444256, -0.03670534864068031, 0....",996
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000","In Hawaii in 1941, a private is cruelly punish...","[-0.08005382120609283, 0.05756460130214691, -0...",997
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,,Several survivors of a torpedoed merchant ship...,"[-0.020807666704058647, 0.05352263152599335, -...",998


In [95]:
from tqdm.auto import tqdm

# We will use batches of 64
batch_size = 64

for i in tqdm(range(0, len(df), batch_size)):
  i_end = min(i + batch_size, len(df))
  # extract batch
  batch = df[i:i_end]
  ids = batch['ids']
  emb = batch['embeddings']
  metadata = batch.drop(['ids', 'embeddings', 'text'], axis=1).to_dict('records')
  # add all to upsert list
  to_upsert = list(zip(ids, emb, metadata))
  _ = index.upsert(to_upsert)

index.describe_index_stats()



  0%|          | 0/16 [00:00<?, ?it/s]

{'dimension': 384,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 1000}},
 'total_vector_count': 1000}

In [94]:
index.delete(ids=['0'])

{}

In [96]:
index.fetch(['0'])

{'namespace': '',
 'vectors': {'0': {'id': '0',
                   'metadata': {'Certificate': 'A',
                                'Director': 'Frank Darabont',
                                'Genre': 'Drama',
                                'Gross': '28,341,469',
                                'IMDB_Rating': 9.3,
                                'Meta_score': 80.0,
                                'No_of_Votes': 2343110.0,
                                'Overview': 'Two imprisoned men bond over a '
                                            'number of years, finding solace '
                                            'and eventual redemption through '
                                            'acts of common decency.',
                                'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg',
                                'Released_Year': '1994',
           

#### Pinecone query

In [97]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [104]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()

In [105]:
responses = index.query(vector=query_vector, top_k = 3, include_metadata=True)

In [106]:
responses

{'matches': [{'id': '761',
              'metadata': {'Certificate': 'U',
                           'Director': 'Mamoru Hosoda',
                           'Genre': 'Animation, Adventure, Comedy',
                           'Gross': ' ',
                           'IMDB_Rating': 7.7,
                           'Meta_score': ' ',
                           'No_of_Votes': 60368.0,
                           'Overview': 'A high-school girl named Makoto '
                                       'acquires the power to travel back in '
                                       'time, and decides to use it for her '
                                       'own personal benefits. Little does she '
                                       'know that she is affecting the lives '
                                       'of others just as much as she is her '
                                       'own.',
                           'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMzA4ZGM1NjYtMjcxY

#### Filter

```
filter = { "genre": {"$eq": "documentary"}, "year": 2019 }
```

In [108]:
query = 'a history of time travel'
query_vector = model.encode(query).tolist()
responses = index.query(vector=query_vector, top_k = 3, include_metadata=True,
                        filter = { "genre": {'$ne': "Animation, Adventure, Comdey"}
                        })


In [109]:
responses

{'matches': [{'id': '761',
              'metadata': {'Certificate': 'U',
                           'Director': 'Mamoru Hosoda',
                           'Genre': 'Animation, Adventure, Comedy',
                           'Gross': ' ',
                           'IMDB_Rating': 7.7,
                           'Meta_score': ' ',
                           'No_of_Votes': 60368.0,
                           'Overview': 'A high-school girl named Makoto '
                                       'acquires the power to travel back in '
                                       'time, and decides to use it for her '
                                       'own personal benefits. Little does she '
                                       'know that she is affecting the lives '
                                       'of others just as much as she is her '
                                       'own.',
                           'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMzA4ZGM1NjYtMjcxY

#### Load Index

In [110]:
pinecone.init(api_key=pinecone_api, environment="us-west1-gcp")

In [111]:
index_2 = pinecone.Index('movies-embeddings')

In [112]:
query = 'a history of an space jorney'
query_vector = model.encode(query).tolist()
responses = index_2.query(vector=query_vector, top_k = 3, include_metadata=True)

In [113]:
responses

{'matches': [{'id': '575',
              'metadata': {'Certificate': 'UA',
                           'Director': 'Theodore Melfi',
                           'Genre': 'Biography, Drama, History',
                           'Gross': '169,607,287',
                           'IMDB_Rating': 7.8,
                           'Meta_score': 74.0,
                           'No_of_Votes': 200876.0,
                           'Overview': 'The story of a team of female '
                                       'African-American mathematicians who '
                                       'served a vital role in NASA during the '
                                       'early years of the U.S. space program.',
                           'Poster_Link': 'https://m.media-amazon.com/images/M/MV5BMzg2Mzg4YmUtNDdkNy00NWY1LWE3NmEtZWMwNGNlMzE5YzU3XkEyXkFqcGdeQXVyMjA5MTIzMjQ@._V1_UX67_CR0,0,67,98_AL_.jpg',
                           'Released_Year': '2016',
                           'Runtime': '127 min',
  